In [ ]:
from neural_map import NeuralMap, tiles
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('datasets/iris.csv', header=None)
df.columns = ['A', 'B', 'C', 'D', 'Type']
l_columns = ['Type']
col = 'Type'

In [ ]:
scaler = MinMaxScaler()
data = scaler.fit_transform(df.drop(l_columns, axis=1).values)

data.shape

In [ ]:
som = NeuralMap(variables=data.shape[1], columns=13, rows=13, metric='cosine', hexagonal=False, toroidal=True)

In [ ]:
som.train(data=data, n_epochs=40)

In [ ]:
som.plot_analysis(data, min_cluster_size=20, attached_values=df[col])

In [ ]:
som.plot_unified_distance_matrix(detailed=True, borders=True)